In [1]:
import pickle
import multiprocessing

from time import time

import gensim
from gensim.models import FastText
from gensim.models.phrases import Phrases, Phraser

print(" setup complete")

 setup complete


## Load Data

In [2]:
with open('../../data/x_train.pickle', 'rb') as handle:
    X_train = pickle.load(handle)
with open('../../data/y_train.pickle', 'rb') as handle:
    y_train = pickle.load(handle)
with open('../../data/x_val.pickle', 'rb') as handle:
    X_val = pickle.load(handle)
with open('../../data/y_val.pickle', 'rb') as handle:
    y_val = pickle.load(handle)

## Variables

In [3]:
maxLen = 150
BATCH_SIZE = 32 #20
MAX_NUM_WORDS = 20000
MAX_SEQ_LEN = 200

## Bigrams

In [4]:
sent = []
for col in ['question', 'question_body', 'answer']:
    words = [row.split() for row in X_train[col]]
    sent = sent + words

In [5]:
sentences = Phrases(sent)

In [6]:
bigram = Phraser(sentences)

In [7]:
sentences = bigram[sent]

## Gensim word to vector

In [8]:
cores = multiprocessing.cpu_count() 
cores

8

In [9]:
len(sentences)

30000

In [10]:
fast_text_model = FastText(min_count=50,
                     window=10,
                     size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=50,
                     workers=cores-1)

In [11]:
#size = 100 
#window = 10
epochs = 50

In [12]:
t = time() 
fast_text_model.build_vocab(sentences, progress_per=10000)
print('time to build vocabulary : ', format(round((time() - t) / 60, 2)))

time to build vocabulary :  0.4


## Training Model

In [13]:
t = time()
fast_text_model.train(sentences, total_examples=fast_text_model.corpus_count, epochs=50, report_delay=1)
print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

Time to train the model: 6.76 mins


In [14]:
pretrained_weights = fast_text_model.wv.syn0
vocab_size, embedding_size = pretrained_weights.shape

C:\Users\achenu\Anaconda3\envs\FAQ\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  """Entry point for launching an IPython kernel.


In [15]:
with open('../../data/fast_text_model_pretrained_weights.pickle', 'wb') as handle:
    pickle.dump(fast_text_model, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [16]:
fast_text_model.most_similar(['python'])

C:\Users\achenu\Anaconda3\envs\FAQ\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[("'python", 0.7581546306610107),
 ('bin_python', 0.6236727237701416),
 ('Python', 0.6218990683555603),
 ('ipython', 0.6028035879135132),
 ('python_exe', 0.5754696726799011),
 ('python_script', 0.5735671520233154),
 ('python_interpreter', 0.541252613067627),
 ('python_site', 0.5158123970031738),
 ('python_org', 0.5104994773864746),
 ('pythonic', 0.5081031322479248)]

In [17]:
# © Axel CHENU 2021